In [105]:
import baltic as bt
import operator
import numpy as np
import copy
tree2 = bt.loadNexus('alleles_spatial_clone_55.mcc', absoluteTime=False) #tree with locations + clones as infered traits

In [83]:
tree2.calc_LBI()

AttributeError: tree instance has no attribute 'calc_LBI'

In [106]:
import operator
import copy
def calc_LBI(tree, attr = 'lb', tau=0.0005, transform = lambda x:x):
    '''.
    traverses the tree in postorder and preorder to calculate the
    up and downstream tree length exponentially weighted by distance.
    then adds them as LBI
    tree -- dendropy tree for whose node the LBI is being computed
    attr	 -- the attribute name used to store the result

    '''
    newtree=copy.deepcopy(tree)
    newtree.postorder_node = sorted(newtree.Objects, key=operator.attrgetter('height'), reverse = True)
    newtree.preorder_internal_node = sorted(newtree.nodes, key = operator.attrgetter('height'), reverse = False)

    # traverse the tree in postorder (children first) to calculate msg to parents
    for node in newtree.postorder_node:
        node.down_polarizer = 0
        node.up_polarizer = 0
        if isinstance(node,bt.node) or node.branchType=='node':
            for child in node.children:
                node.up_polarizer += child.up_polarizer
        bl =  node.length/tau
        node.up_polarizer *= np.exp(-bl)
        node.up_polarizer += tau*(1-np.exp(-bl))

    # traverse the tree in preorder (parents first) to calculate msg to children
    for node in newtree.preorder_internal_node:
        for child1 in node.children:
            child1.down_polarizer = node.down_polarizer
        for child2 in node.children:
            if child1!=child2:
                child1.down_polarizer += child2.up_polarizer

        bl =  child1.length/tau
        child1.down_polarizer *= np.exp(-bl)
        child1.down_polarizer += tau*(1-np.exp(-bl))

    # go over all nodes and calculate the LBI (can be done in any order)
    for node in newtree.postorder_node:
        tmp_LBI = node.down_polarizer
        if isinstance(node,bt.node) or node.branchType=='node':
            for child in node.children:
                tmp_LBI += child.up_polarizer
    #node.__setattr__(attr, transform(tmp_LBI))
        setattr(node, attr, transform(tmp_LBI))
    return newtree

In [107]:
ttree = calc_LBI(tree2)

In [108]:
for k in ttree.Objects:
    print k.lb

0.00100104137618
0.00150000733033
0.0015
0.00150000000001
0.00150000290264
0.00149917163843
0.000500002915326
0.00108345537206
0.000503198123355
0.000623121776112
0.000500000000007
0.0005
0.00150000000819
0.00175643067689
0.00150490029277
0.00151266156601
0.000504824228257
0.00154200672126
0.000507405361327
0.00152671486208
0.000536108218488
0.000504991738
0.00141320576817
0.0005
0.000539630496576
0.00176304441765
0.00174982683567
0.0015180963204
0.000500001021841
0.00151946071763
0.0012742841055
0.000504616376266
0.000589702958333
0.000518081289578
0.00165030149601
0.000749826129221
0.000569673594173
0.00151995675901
0.00150014498338
0.000500144978173
0.0015007461856
0.00149945552283
0.000500749085929
0.000500646913885
0.000500000005201
0.00140832571998
0.000519817307924
0.000551740070011
0.00150089316522
0.00151618975989
0.00150079979625
0.000500799796202
0.00149999911825
0.000500000000051
0.000500000440902
0.00153083172328
0.000500000000175
0.00175738303833
0.00176827104493
0.001788